In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [3]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
print(es_client.info())

{'name': '1c42be122103', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'gderiaSRRIaKv_UerXz6yg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}


index_name = "module-1-homework"

es_client.indices.create(index=index_name, body=index_settings)

/var/folders/z3/z42199yd22n5kx0lwyx0_nqc0000gn/T/ipykernel_35343/52094710.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'module-1-homework'})

In [6]:
from tqdm.auto import tqdm

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [56]:
def elastic_search_for_hw(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index="module-1-homework", body=search_query)
    return response

In [57]:
def get_score_from_qs_and_as(query):
    response = elastic_search_for_hw(query)
    return response["hits"]["hits"][0]["_score"]

get_score_from_qs_and_as("How do execute a command on a Kubernetes pod?")

/var/folders/z3/z42199yd22n5kx0lwyx0_nqc0000gn/T/ipykernel_35343/3853175986.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index="module-1-homework", body=search_query)


44.50556

In [58]:
def es_retrieve_qs_and_a(query):
    response = elastic_search_for_hw(query)
    result_docs = [
        {
            "question": hit["_source"].get("question", ""),
            "text": hit["_source"].get("text", "")
        }
        for hit in response["hits"]["hits"]
    ]

    return result_docs

In [59]:
es_retrieve_qs_and_a("How do I execute a command in a running docker container?")

/var/folders/z3/z42199yd22n5kx0lwyx0_nqc0000gn/T/ipykernel_35343/3853175986.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index="module-1-homework", body=search_query)


[{'question': 'How do I debug a docker container?',
  'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)'},
 {'question': 'How do I copy files from my local machine to docker container?',
  'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani"},
 {'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'text': 'You can copy files from your loc

In [60]:
# q4
es_retrieve_qs_and_a("How do copy a file to a Docker container?")[2]['question']

/var/folders/z3/z42199yd22n5kx0lwyx0_nqc0000gn/T/ipykernel_35343/3853175986.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index="module-1-homework", body=search_query)


'How do I copy files from a different folder into docker container’s working directory?'

In [61]:
#q5
context_template = """
Q: {question}
A: {text}
""".strip()
query = "How do I execute a command in a running docker container?"
answers = es_retrieve_qs_and_a(query)

formatted_context = ''
for each in answers:
    formatted_context += context_template.format(question=each['question'], text=each['text'])

/var/folders/z3/z42199yd22n5kx0lwyx0_nqc0000gn/T/ipykernel_35343/3853175986.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index="module-1-homework", body=search_query)


In [64]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=formatted_context)
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/p

In [65]:
len(prompt)

2706

In [66]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [68]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [70]:
#q6
len(encoding.encode(prompt))

619